# 数据集介绍

数据集类型是微博情感分类 

来源https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/weibo_senti_100k/intro.ipynb

# 导入包

In [1]:
import sys,os
import pandas as pd
import random
import numpy as np
from nlp_basictasks.tasks import cls
from nlp_basictasks.evaluation import clsEvaluator
from nlp_basictasks.readers.cls import getExamplesFromData
import nlp_basictasks
print(nlp_basictasks.__version__)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
0.1.4


# 获取数据

In [2]:
data_path='/data/nfs14/nfs/aisearch/asr/xhsun/datasets/weibo_senti_100k.csv'
pd_all = pd.read_csv(data_path)

print('评论数目（总体）：%d' % pd_all.shape[0])
print('评论数目（正向）：%d' % pd_all[pd_all.label==1].shape[0])
print('评论数目（负向）：%d' % pd_all[pd_all.label==0].shape[0])

评论数目（总体）：119988
评论数目（正向）：59993
评论数目（负向）：59995


In [3]:
pd_all.sample(20)

,label,review
3032,1,霾都欢迎你再来。。。。。。[嘻嘻]
40365,1,据说花名叫＂大烟桐＂，不知是否准确。 //@刘道勇oO:继续求助//@医生passerby:...
2413,1,当马栏坡坡姐遇上新加坡坡姐[嘻嘻] ！笑料一波又一波，颇有看点！@谢娜 这个动作@孙燕姿 要...
84373,0,看着也还行//@小四爱世博: 李伯祥，赵伟洲我都爱看，刘兰芳作为开奖嘉宾，不解??? //@...
70913,0,我真有[失望][耶]//@全球奇事趣闻:我希望我可以选择性失忆。[心]
93627,0,人在中山，对话厉无畏先生。来去匆匆，没吃到中山乳鸽，有些伤感。[失望]
3085,1,哥，你一点儿不老，在我眼中你就是一帅小伙儿[嘻嘻]
68049,0,"傻逼联通3G,下班高峰期地铁一号线，十号线完全上不了网，就尼玛会吹3G速度比移动快，移动在地..."
24206,1,回复@很靠谱的叁个六:哈哈，给你留一本呢。我签的名，你看咋样？[哈哈] //@很靠谱的叁个六...
54556,1,[爱你]哇！超级浪漫啊！


## 打乱数据集，同时划分训练、验证

In [4]:
print(len(pd_all))
random_idx=np.random.permutation(len(pd_all))

119988


In [5]:
#时间关系仅取前20000个作为训练
sentences=pd_all['review'].values[random_idx].tolist()[:20000]
labels=pd_all['label'].values[random_idx].tolist()[:20000]
print(len(sentences),len(labels))
random_idx=np.random.permutation(len(sentences))

20000 20000


In [6]:
label2id={'0':0,'1':1}
dev_ratio=0.2#训练集的20%作为验证
dev_nums=int(len(sentences)*dev_ratio)
train_nums=len(sentences)-dev_nums
print(dev_nums)
train_sentences=sentences[:train_nums]
train_labels=labels[:train_nums]
dev_sentences=sentences[-dev_nums:]
dev_labels=labels[-dev_nums:]
train_examples,max_seq_len=getExamplesFromData(sentences=train_sentences,labels=train_labels,label2id=label2id,mode='train',return_max_len=True)
dev_examples=getExamplesFromData(sentences=dev_sentences,labels=dev_labels,label2id=label2id,mode='dev')

2021-08-24 19:42:44 - INFO - getExamplesFromData - 128 : *****************************Logging some train examples*****************************
2021-08-24 19:42:44 - INFO - getExamplesFromData - 129 : Total train nums is : 16000
2021-08-24 19:42:44 - INFO - getExamplesFromData - 132 : 
体育馆，爽啊！！[哈哈][哈哈][哈哈][哈哈][哈哈][哈哈][哈哈][哈哈]	1
2021-08-24 19:42:44 - INFO - getExamplesFromData - 132 : 
正在看@蒋蒋韵  主笔的@韩庚 专访，欣赏ing[鼓掌]@韩庚粉丝团-江苏 @韩庚吧 @韩庚歌友会官方 @王者韩庚论坛   可买这期杂志看看[呵呵]	1
2021-08-24 19:42:44 - INFO - getExamplesFromData - 132 : 
我们这边都不纪念了[晕]其实很可惜，很好回顾历史，推广战地旅游的机会[汗]	0
2021-08-24 19:42:44 - INFO - getExamplesFromData - 132 : 
[鼓掌] //@峤九一://@南多酱:壮哉我大成都！{只转不看，看了要哭！[可怜]} //@Just__We://@枣子_求警花给条活路:壮哉我大天府吃货帝国！	1
2021-08-24 19:42:44 - INFO - getExamplesFromData - 132 : 
这次的周末驾到去中蒙边境[嘻嘻]	1
2021-08-24 19:42:44 - INFO - getExamplesFromData - 128 : *****************************Logging some dev examples*****************************
2021-08-24 19:42:44 - INFO - getExamplesFromData - 129 : Total dev nums is : 40

4000


# 定义路径加载模型

In [8]:
#max_seq_len就是训练集中最长的句子长度
model_path='/data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/'
print(max_seq_len)
max_seq_len=min(512,max_seq_len)
cls_model=cls(model_path=model_path,label2id=label2id,max_seq_length=max_seq_len,device='cpu')

2021-08-24 19:43:49 - INFO - __init__ - 48 : The label2id is
 {"0": 0, "1": 1}
2021-08-24 19:43:49 - INFO - __init__ - 58 : Loading model from /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/, which is from huggingface model
2021-08-24 19:43:49 - INFO - get_config_dict - 177 : loading configuration file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/config.json
2021-08-24 19:43:49 - INFO - from_pretrained - 404 : loading bert model file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/
2021-08-24 19:43:49 - INFO - from_pretrained - 423 : BertConfig has been loaded from /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/config.json


256


2021-08-24 19:43:51 - INFO - from_pretrained - 125 : loading vocabulary file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/vocab.txt


In [9]:
from torch.utils.data import DataLoader
batch_size=32
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

# 定义evaluator

In [11]:
evaluator=clsEvaluator(sentences=dev_sentences,label_ids=dev_labels,write_csv=False,label2id=label2id)

2021-08-24 19:44:28 - INFO - __init__ - 28 : label2id like : {'0': 0, '1': 1}
2021-08-24 19:44:28 - INFO - __init__ - 39 : The number of 0 in dataset is 2020
2021-08-24 19:44:28 - INFO - __init__ - 39 : The number of 1 in dataset is 1980
2021-08-24 19:44:28 - INFO - __init__ - 45 : Evalautor sentence like : 

2021-08-24 19:44:28 - INFO - __init__ - 47 : [衰] //@中原网:【图片中的“海市蜃楼”系周边建筑】有结果啦，虚惊一场，有多位网友向小编反映，下午的时候这个点经过该路口，并未发现有“海市蜃楼”现象出现，图片上的“海市蜃楼”应该是对面的建筑（经现场核实，系附近一楼盘）反射在玻璃板上，正好被拍下来了，拍摄者当时可能坐在公交车里拍的。	0

2021-08-24 19:44:28 - INFO - __init__ - 47 : @一辰 ，尼玛太有才了[cai正呀] [哈哈] @隋杨	1

2021-08-24 19:44:28 - INFO - __init__ - 47 : 小伙伴儿们现在拿起手机，打开微博客户端-广场，双节有惊喜呦！[吃元宵][moc亲吻]//@兰亦兰: [蜡烛][泪]//@我不是蛐蛐: [哈哈] //@郭宏:转发微博	1

2021-08-24 19:44:28 - INFO - __init__ - 47 : 我们饮中八仙也郑重承诺 剩下的画展 我们必须都去 哈哈//@大匠之门李笑天: 回复@八年小散:我郑重诚诺。要在中国美术办它10次画展[嘻嘻][嘻嘻] //@八年小散:还有听见此话的吗？ //@大匠之门李笑天:[嘻嘻][嘻嘻][威武] //@郭月晨:这个我听见了 //@苗雪珊: [可爱]真滴嘛	1

2021-08-24 19:44:28 - INFO - __init__ - 47 : [蜡烛][蜡烛][蜡烛]求报复社会的人们放过无辜老百姓吧[伤心][伤心][伤心]	0



# 训练模型
- is_pairs用来指明是单句子分类任务还是双句子分类任务
- output_path是保存模型的路径

In [12]:
cls_model.fit(is_pairs=False,train_dataloader=train_dataloader,evaluator=evaluator,output_path="/data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/")

2021-08-24 19:45:35 - INFO - fit - 102 : 当前是单句子分类任务
2021-08-24 19:45:35 - INFO - fit - 113 : 一个epoch 下，每隔100个step会输出一次loss，每隔250个step会评估一次模型


2021-08-24 19:45:43 - INFO - __call__ - 63 : ClsEvaluator: Evaluating the model on  dataset in epoch 0 after 1 steps:
2021-08-24 19:45:43 - INFO - predict - 221 : 当前是单句子分类任务预测


2021-08-24 19:49:18 - INFO - __call__ - 70 : Accuracy: 0.436
2021-08-24 19:49:18 - INFO - __call__ - 82 : 0	precision : 0.450293, recall : 0.529205,  f1 score : 0.481653
2021-08-24 19:49:18 - INFO - __call__ - 82 : 1	precision : 0.415127, recall : 0.340907,  f1 score : 0.369487
2021-08-24 19:49:18 - INFO - __call__ - 93 : AUC: 0.404


2021-08-24 19:49:20 - INFO - save_pretrained - 509 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-24 19:49:20 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-24 19:49:20 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-24 19:49:20 - INFO - fit - 188 : In epoch 0, training_step 0, the eval score is 0.403772502250225, previous eval score is -9999999, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/
2021-08-24 20:00:12 - INFO - fit - 166 : Epoch : 0, train_step : 100/500, loss_value : 0.341104484423995 
2021-08-24 20:10:16 - INFO - fit - 166 : Epoch : 0, train_step : 200/500, loss_value : 0.06391732832184062 
2021-08-24 20:15:35 - INFO - __call__ - 63 : ClsEvaluator: Evaluating the model on  dataset in epoch 0 after 251 steps:
2021-08-24 20:15:35 - INFO - predict - 221 : 当前是单句子分

2021-08-24 20:18:48 - INFO - __call__ - 70 : Accuracy: 0.981
2021-08-24 20:18:48 - INFO - __call__ - 82 : 0	precision : 0.965529, recall : 0.998510,  f1 score : 0.976769
2021-08-24 20:18:48 - INFO - __call__ - 82 : 1	precision : 0.998425, recall : 0.963631,  f1 score : 0.975747
2021-08-24 20:18:48 - INFO - __call__ - 93 : AUC: 0.998


2021-08-24 20:18:55 - INFO - save_pretrained - 509 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-24 20:18:55 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-24 20:18:55 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-24 20:18:55 - INFO - fit - 188 : In epoch 0, training_step 250, the eval score is 0.9983108310831084, previous eval score is 0.403772502250225, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/
2021-08-24 20:23:54 - INFO - fit - 166 : Epoch : 0, train_step : 300/500, loss_value : 0.04959781204350293 


KeyboardInterrupt: 

**时间关系暂停训练**

# 测试模型

In [16]:
predict_probs=cls_model.predict(is_pairs=False,dataloader=['这孩子真可爱','这人看起来像傻子似的'])
print(predict_probs)
id2label={id_:label for label,id_ in label2id.items()}
predict_tags=[id2label[id_] for id_ in np.argmax(predict_probs,axis=1)]
print(predict_tags)

2021-08-24 20:29:10 - INFO - predict - 221 : 当前是单句子分类任务预测



[[0.00504928 0.9949508 ]
 [0.6484326  0.35156742]]
['1', '0']
